In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-nfl-predictions/nfl_elo.csv


In [2]:
games = pd.read_csv('/kaggle/input/2021-nfl-predictions/nfl_elo.csv')
games.pop('date')
games.pop('season')
games.pop('neutral')
games.pop('playoff')
games.pop('team1')
games.pop('team2')
games.pop('qb1_value_post')
games.pop('qb2_value_post')
games.pop('qbelo1_post')
games.pop('qbelo2_post')
games.pop('quality')
games.pop('total_rating')
games.pop('elo1_post')
games.pop('elo2_post')
games.pop('qb1')
games.pop('qb2')
games.pop('importance')
games.pop('elo_prob2')
games.pop('qbelo_prob2')
games = games.iloc[2200:,:]
games['scorecard'] = games['score1'].astype(str) + '-' + games['score2'].astype(str)
games.pop('score1')
games.pop('score2')
games

,elo1_pre,elo2_pre,elo_prob1,qbelo1_pre,qbelo2_pre,qb1_value_pre,qb2_value_pre,qb1_adj,qb2_adj,qbelo_prob1,qb1_game_value,qb2_game_value,scorecard
2200,1705.868000,1374.898000,0.907155,1708.412782,1372.855800,11.529330,25.687451,5.887357,-0.760251,0.907971,27.328090,-25.250947,35-3
2201,1453.682000,1647.227000,0.323012,1450.814706,1645.463657,46.537729,42.526623,20.393099,20.735110,0.290656,252.494661,109.816382,38-27
2202,1665.617000,1487.838000,0.801795,1660.007781,1487.711872,37.426853,-3.498000,15.976932,-21.671167,0.821681,214.692789,92.967761,45-7
2203,1589.174000,1462.687000,0.750689,1590.849355,1463.375342,23.355407,27.071473,9.927229,11.256738,0.741393,439.954968,-16.783853,65-24
2204,1711.370000,1474.759000,0.850205,1713.863175,1476.052720,13.109206,9.890100,6.382927,-12.022796,0.857840,-160.486990,-8.304852,7-9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17090,1681.797158,1632.341105,0.659005,1649.547292,1615.165080,249.882409,179.870436,11.456236,-0.623739,0.681440,132.180809,379.027146,27-30
17091,1699.470145,1683.332367,0.614689,1687.450699,1656.910634,273.771767,275.630099,20.577067,18.202487,0.652720,622.042187,434.101420,42-36
17092,1713.932369,1569.498961,0.769519,1700.324756,1559.717895,308.598809,190.637925,36.961734,24.405309,0.810417,134.683170,153.972640,24-27
17093,1651.615346,1621.465562,0.633608,1635.265484,1631.632019,199.786107,117.398901,9.365284,-11.464001,0.636093,248.504596,197.148060,20-17


In [3]:
def win_or_lose(score):
    dash = score.index('-')
    team1 = int(score[:dash])
    team2 = int(score[dash+1:])
    if team2 == 0 or team1 / team2 <= 1:
        return 0
    else:
        return 1

In [4]:
games['win?'] = games['scorecard'].apply(win_or_lose)
games.pop('scorecard')
y = games['win?']
games.pop('win?')
y

2200     1
2201     1
2202     1
2203     1
2204     0
        ..
17090    0
17091    1
17092    0
17093    1
17094    0
Name: win?, Length: 14895, dtype: int64

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
        games, y, test_size=0.2, random_state=1234
    )

In [6]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(penalty='l1', tol=0.001, solver='liblinear', max_iter=1500, multi_class='ovr')

model.fit(X_train, y_train)
y_pred = model.predict_proba(X_test)
y_pred = y_pred[:,1]

from sklearn.metrics import accuracy_score
accuracy_score(y_test,np.round(y_pred))

0.7472306143001007